[Реализация](https://github.com/ttk592/spline) кубических сплайнов с настройкой граничных условий вида `not_a_knot` совпадает по результатам с реализацией `scipy.interpolate.interp1d(t, x, kind='cubic')`:
- отклонение порядка среднеквадратичное `1e-10`, максимальное `1e-5`

In [13]:
import numpy as np
import json

from scipy.interpolate import interp1d

trajectory = json.load(open("trajectory.json"))

T = [p['t'] for p in trajectory]
X = [p['xyz'][0] for p in trajectory]
Y = [p['xyz'][1] for p in trajectory]
Z = [p['xyz'][2] for p in trajectory]

x = interp1d(T, X, kind='cubic')
y = interp1d(T, Y, kind='cubic')
z = interp1d(T, Z, kind='cubic')

num_segments = 50

t = np.linspace(0, 1, 50 * len(T))

X_hat = x(t)
Y_hat = y(t)
Z_hat = z(t)

trajectory_scipy = [{"t": F[0], "xyz": [F[1],F[2],F[3]]} for F in zip(t, X_hat, Y_hat, Z_hat)]

json.dump(trajectory_scipy, open("trajectory_scipy.json", "w"), indent=4)

Аналогичная интерполяция `spline.h`

In [25]:
spline = json.load(open("spline.json"))

Ts = [p['t'] for p in spline]
Xs = [p['xyz'][0] for p in spline]
Ys = [p['xyz'][1] for p in spline]
Zs = [p['xyz'][2] for p in spline]

In [28]:
dt = [d[0] - d[1] for d in zip(t, Ts)]
dx = [d[0] - d[1] for d in zip(X_hat, Xs)]
dy = [d[0] - d[1] for d in zip(Y_hat, Ys)]
dz = [d[0] - d[1] for d in zip(Z_hat, Zs)]

Отклонение
- суммарное

In [36]:
sum(dx) + sum(dy) + sum(dz)

0.0004774319781428874

- среднее

In [37]:
(sum(dx) + sum(dy) + sum(dz)) / (len(dx) + len(dy) + len(dz))

2.1219199028572773e-07

- максимальное

In [39]:
max(map(abs, dx+dy+dz))

4.9969773675329066e-05

- среднеквадратичное

In [46]:
sum(list(map(lambda x: x**2, dx+dy+dz))) / len(dx+dy+dz)

4.808634924079471e-10